In [2]:
# Dependencies
import requests
import json
import pandas as pd
from pprint import pprint
from config import client_id, client_secret

Following instructions from: https://stmorse.github.io/journal/spotify-api.html
AND from Spotify documentation:
https://developer.spotify.com/documentation/general/guides/authorization-guide/#list-of-scopes

## Authorization flow

Step 1: Request authorization to access data. No code. 
- Example Grant access URL:
https://accounts.spotify.com/authorize?response_type=code&client_id=5d67ef47a63a42f08feb4c824ee174f7&scope=user-top-read&redirect_uri=https%3A%2F%2Fgoogle.com%2Fcallback&show_dialog=true

Once accept, data can now be requested.

Note: if changing `scope`, this step and step 2 need to be repeated.
Scope for the following:
- User's Top Artists and Tracks: `user-top-read`
- List of a User's Playlists: `playlist-read-private`

In [1]:
# Once access is granted, copy the code from the URL
code = 'AQCGwjO4KA1CRDlyaBSlCaHi-FuJZDY40Am7MeSNC0776WG3b1MQcpMyUGHDfWKrTYxW1Mu9ntazWBUieq18fCmd5jRj9skEIfLcH7FycauBvFOfLx2WMV5i1n4BkBmAXy5p5GlvFBnJztdu2WxsFwox3R_q556VP9w8b6yDDDFNAbFISmPHyEGNLWCMVA'

Step 2: Request to get refresh and access token
- Make POST request to Spotify on /api/token endpoint
- Require parameters specfied accordingly.

In [3]:
# POST - REQUEST BODY PARAMETER, skipping header parameter using alternative way
request_body_parameter = {
    'grant_type': 'authorization_code',
    'code': code,
    'redirect_uri': 'https://google.com/callback',
    'client_id': client_id,
    'client_secret': client_secret,
}

In [4]:
# Pass an access token - POST a request with client credentials and save part of response
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST - REQUEST BODY PARAMETER
auth_response = requests.post(AUTH_URL, request_body_parameter)

# convert the response to JSON
auth_response_data = auth_response.json()

# Try print it out to get refresh_token
print(auth_response_data)

{'access_token': 'BQA9M7mZ9_-7uSpTnoPPm8-cJ6ugifRwRicjs6IADPwlMUJfeYpHz-KUUqFi_oXJST-lxCWtUaiwcmDD-DUcI8Pc23GoYYKXdcrl6x_RcB8krJyAlbD38wxEBXlq8yYPVGGZdwk_BDQAwO53IuM', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQBpLFi-p9cogEtqin0A9EfjxnnrK_ROJ8-o8LRzBjl871WSKMIhVxMdqazLZCF0XCs-ShdqSe2buLdbLza2qvlIpbjqXWfkGDJFBZAQ4oh9DyZp0TSrknw4A12hIevUBYM', 'scope': 'user-top-read'}


In [5]:
# Save the access token
access_token = auth_response_data['access_token']

# Save the refresh token
refresh_token = auth_response_data['refresh_token']

Step 3: Use access token to access data of interests(?)
- Note: when access token expires, it can't be used anymore.

#### Get a User's Top Artists and Tracks

In [6]:
# Form GET request to API server with access_token in the header
headers = {
    'Authorization': f'Bearer {access_token}'
}

# base URL of all Spotify API endpoints
base_url = 'https://api.spotify.com/v1/'

# user's top track
top_track = 'me/top/tracks'
time_range = 'time_range=medium_term'
limit = 'limit=50'
offset = 'offset=0'

### If token expires, go to Step 4 and run those code and run the code below

In [7]:
# Perfrom a request for data
track_json = requests.get(base_url + top_track + '?' + time_range + '&' + limit + '&' + offset
                          , headers=headers).json()

In [8]:
# Save outout to text file to go parse later...
with open('top_tracks.json', 'w') as outfile:
    json.dump(track_json, outfile, indent=2)

In [28]:
# Write a function that uses map to take the data in array (either array of array or array of object, etc) 
## and apply it in each item in an array - Return me an array from an array with only the pieces I want

# Instead of define a bunch of empty lists and create a loop to append each item in to each list, this is much cleaner and more efficient

def track_func(track):
#artists = [artist['name'] for artist in track['artists']]
# artist['name']
# { 'id': artist['id'], 'name': artist['name'] }
    artists_id = [  artist['id']  for artist in track['artists'] ]
    artists_name = [ artist['name']  for artist in track['artists'] ]
    album_id = track['album']['id']
    album_name = track['album']['name']
    release_date = track['album']['release_date']
    song_id = track['id']
    title = track['name']
    popularity = track['popularity']
    duration_ms = track['duration_ms']
    song_url = track['external_urls']['spotify']

    return {
        'song_id': song_id,
        'title': title,
        'artists_id': str(artists_id)[2:-2],
        'artists_name': str(artists_name)[2:-2],
        'album_id': album_id,
        'album_name':album_name,
        'release_date':release_date,
        'duration_min': round(duration_ms/60000, 2),
        'popularity': popularity,
        'song_url': song_url
        
    }
# Use map to give a list iterator
track_list_iterator = map(track_func, track_json['items'])
# Turn iterator into a list
track_list = list(track_list_iterator)
track_list

[{'song_id': '06XQvnJb53SUYmlWIhUXUi',
  'title': '怪物',
  'artists_id': '64tJ2EAv1R6UaZqc4iOCyj',
  'artists_name': 'YOASOBI',
  'album_id': '41HUxKwnbrg8IdelmMibj9',
  'album_name': '怪物',
  'release_date': '2021-01-06',
  'duration_min': 3.43,
  'popularity': 82,
  'song_url': 'https://open.spotify.com/track/06XQvnJb53SUYmlWIhUXUi'},
 {'song_id': '6vc3GcAg27Z9iknbjBVZtH',
  'title': '世界とかくれんぼ',
  'artists_id': "6Na6YdJLEeBDbP1bZv6q7a', '2EZcXq4boUMbuGRsSWbjeC",
  'artists_name': "国江田計 (CV: 阿部敦)', '都築潮 (CV: 川原慶久)",
  'album_id': '7gU3VRaRiSBPaynGeLjhTJ',
  'album_name': '世界とかくれんぼ',
  'release_date': '2020-12-04',
  'duration_min': 3.91,
  'popularity': 22,
  'song_url': 'https://open.spotify.com/track/6vc3GcAg27Z9iknbjBVZtH'},
 {'song_id': '4kFY1RxOo6Wd8hgVL3t6Rb',
  'title': 'Phantom Joke',
  'artists_id': '449AEgfeOxqAuRn0uX6l3u',
  'artists_name': 'UNISON SQUARE GARDEN',
  'album_id': '6Rm9rTuxsvukGVowLHP93c',
  'album_name': 'UNISON SQUARE GARDEN The 1st Collection for streaming',


In [29]:
# Convert the array  of data into a dataframe
top_tracks_df = pd.DataFrame(track_list)
top_tracks_df

,song_id,title,artists_id,artists_name,album_id,album_name,release_date,duration_min,popularity,song_url
0,06XQvnJb53SUYmlWIhUXUi,怪物,64tJ2EAv1R6UaZqc4iOCyj,YOASOBI,41HUxKwnbrg8IdelmMibj9,怪物,2021-01-06,3.43,82,https://open.spotify.com/track/06XQvnJb53SUYml...
1,6vc3GcAg27Z9iknbjBVZtH,世界とかくれんぼ,"6Na6YdJLEeBDbP1bZv6q7a', '2EZcXq4boUMbuGRsSWbjeC","国江田計 (CV: 阿部敦)', '都築潮 (CV: 川原慶久)",7gU3VRaRiSBPaynGeLjhTJ,世界とかくれんぼ,2020-12-04,3.91,22,https://open.spotify.com/track/6vc3GcAg27Z9ikn...
2,4kFY1RxOo6Wd8hgVL3t6Rb,Phantom Joke,449AEgfeOxqAuRn0uX6l3u,UNISON SQUARE GARDEN,6Rm9rTuxsvukGVowLHP93c,UNISON SQUARE GARDEN The 1st Collection for st...,2019-10-11,4.07,52,https://open.spotify.com/track/4kFY1RxOo6Wd8hg...
3,4EPXOp2cwHlYiOpz2YQWYS,スターダスト,1xwuTSJVlnBEOecAEHGRfY,ITOWOKASHI,178MyVBgrjKDFvwYclbee6,中央突破,2017-06-21,4.52,27,https://open.spotify.com/track/4EPXOp2cwHlYiOp...
4,3REf7M4gfHUaS8SAJrrLG7,ARK OF MEMORIES,3i7tuzRfutJGAUowspOM1f,Kashitaro Ito,3ewKbf1sUhzYyGSn8LkpHe,ENDING THEME FROM TV SERIES ”DECADENCE”,2020-07-29,4.20,34,https://open.spotify.com/track/3REf7M4gfHUaS8S...
5,6IP3GvRj6Rc65rly1rIHq5,レディメイド,6mEQK9m2krja6X1cfsAjfl,Ado,7hy4zddDO6zOMQxRipqQry,レディメイド,2020-12-24,4.07,65,https://open.spotify.com/track/6IP3GvRj6Rc65rl...
6,62GRSSM4SdDSp5MXgWdnah,Calc.,3i7tuzRfutJGAUowspOM1f,Kashitaro Ito,5VfxZpps2s3yibA445smqp,一意専心,2014-01-22,3.77,37,https://open.spotify.com/track/62GRSSM4SdDSp5M...
7,6EzZn96uOc9JsVGNRpx06n,うっせぇわ,6mEQK9m2krja6X1cfsAjfl,Ado,5msh7Lspyh1jGbDM2BxwAy,うっせぇわ,2020-10-23,3.42,79,https://open.spotify.com/track/6EzZn96uOc9JsVG...
8,3bnSwZS7PJOCjqZF38iBdZ,rebirthday,3i7tuzRfutJGAUowspOM1f,Kashitaro Ito,5VfxZpps2s3yibA445smqp,一意専心,2014-01-22,5.22,18,https://open.spotify.com/track/3bnSwZS7PJOCjqZ...
9,5meZvaHT6pQx4AZP1f8lN3,カナデアイ,1xwuTSJVlnBEOecAEHGRfY,ITOWOKASHI,178MyVBgrjKDFvwYclbee6,中央突破,2017-06-21,3.89,48,https://open.spotify.com/track/5meZvaHT6pQx4AZ...


Step 4: When access token expires, refresh token to the rescue (from step 2)

In [ ]:
# Base 64 encoded string that contains the client ID and client secret key.
import base64

def encode_base64(message):
    message_bytes = message.encode('ascii')
    base64_bytes = base64.b64encode(message_bytes)
    base64_message = base64_bytes.decode('ascii') 
    return base64_message
#     return str(base64_bytes, "utf-8")
    
print(encode_base64(f'{client_id}:{client_secret}'))

In [ ]:
# Get the new refresh access token and update - only when request fails 
def refresh_access_token():   
    # build body parameter
    body = {
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token
    }
    
    # base64 string from above encode_base64 function
    auth_base64 = encode_base64(f'{client_id}:{client_secret}')
    
    # build header parameter
    header = {
        'Authorization': f'Basic {auth_base64}'
    }
    
    request_access_token = requests.post(AUTH_URL, body, headers=header)
    request_access_token_data = request_access_token.json()
    print(request_access_token_data)
    # Update access_token (for step 3)
    return request_access_token_data['access_token']

In [ ]:
# Update access token from Step 3
access_token = refresh_access_token()

# Update headers
headers = {'Authorization': f'Bearer {access_token}'}

# Future Update
- Save the current time + `expires_in` into a variable, `expire_time`.
- Evey time you make a request for data, add a line of code that checks "Is my current time > `expire_time`?"
- If yes, refresh access token and update headers (step 4)